In [1]:
println("Start")

Intitializing Scala interpreter ...

Spark Web UI available at http://LAPTOP-D4GN0MQF:4042
SparkContext available as 'sc' (version = 2.3.0, master = local[*], app id = local-1573069627261)
SparkSession available as 'spark'


Start


# 1 - Sum consecutive value by Seq

In [2]:
import org.apache.spark.sql.expressions.Window

import org.apache.spark.sql.expressions.Window


In [3]:
val df = Seq(
  (1, 10, 0),
  (1, 11, 1),
  (1, 13, 1),
  (1, 16, 1),
  (1, 20, 0),
  (1, 21, 0),
  (1, 22, 1),
  (1, 25, 1),
  (1, 27, 1),
  (1, 29, 1),
  (1, 30, 0),
  (1, 32, 1),
  (1, 34, 1),
  (1, 35, 1),
  (1, 38, 0)).toDF("Category", "Value", "Sequences")
df.show

+--------+-----+---------+
|Category|Value|Sequences|
+--------+-----+---------+
|       1|   10|        0|
|       1|   11|        1|
|       1|   13|        1|
|       1|   16|        1|
|       1|   20|        0|
|       1|   21|        0|
|       1|   22|        1|
|       1|   25|        1|
|       1|   27|        1|
|       1|   29|        1|
|       1|   30|        0|
|       1|   32|        1|
|       1|   34|        1|
|       1|   35|        1|
|       1|   38|        0|
+--------+-----+---------+



df: org.apache.spark.sql.DataFrame = [Category: int, Value: int ... 1 more field]


assing each row unique id

In [4]:
val zipped = df.withColumn("zip", monotonically_increasing_id())
zipped.show

+--------+-----+---------+---+
|Category|Value|Sequences|zip|
+--------+-----+---------+---+
|       1|   10|        0|  0|
|       1|   11|        1|  1|
|       1|   13|        1|  2|
|       1|   16|        1|  3|
|       1|   20|        0|  4|
|       1|   21|        0|  5|
|       1|   22|        1|  6|
|       1|   25|        1|  7|
|       1|   27|        1|  8|
|       1|   29|        1|  9|
|       1|   30|        0| 10|
|       1|   32|        1| 11|
|       1|   34|        1| 12|
|       1|   35|        1| 13|
|       1|   38|        0| 14|
+--------+-----+---------+---+



zipped: org.apache.spark.sql.DataFrame = [Category: int, Value: int ... 2 more fields]


make range from zero to the next zero

In [5]:
val categoryWindow = Window.partitionBy("Category").orderBy($"zip")
val groups = zipped
             .filter($"Sequences" === 0)
             .withColumn("rangeEnd", lead($"zip",1).over(categoryWindow))
             .withColumnRenamed("zip", "rangeStart")
groups.show(false)

+--------+-----+---------+----------+--------+
|Category|Value|Sequences|rangeStart|rangeEnd|
+--------+-----+---------+----------+--------+
|1       |10   |0        |0         |4       |
|1       |20   |0        |4         |5       |
|1       |21   |0        |5         |10      |
|1       |30   |0        |10        |14      |
|1       |38   |0        |14        |null    |
+--------+-----+---------+----------+--------+



categoryWindow: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@665afaa9
groups: org.apache.spark.sql.DataFrame = [Category: int, Value: int ... 3 more fields]


assing range for each unit

In [6]:
val joinCondition = ($"units.zip" > $"groups.rangeStart").and($"units.zip" < $"groups.rangeEnd")

joinCondition: org.apache.spark.sql.Column = ((units.zip > groups.rangeStart) AND (units.zip < groups.rangeEnd))


In [7]:
val unitsByRange = zipped
                    .filter($"Sequences" === 1).alias("units")
                    .join(groups.alias("groups"), joinCondition, "left")
                    .select("units.Category", "units.Value", "groups.rangeStart")

unitsByRange: org.apache.spark.sql.DataFrame = [Category: int, Value: int ... 1 more field]


In [8]:
unitsByRange.show

+--------+-----+----------+
|Category|Value|rangeStart|
+--------+-----+----------+
|       1|   11|         0|
|       1|   13|         0|
|       1|   16|         0|
|       1|   22|         5|
|       1|   25|         5|
|       1|   27|         5|
|       1|   29|         5|
|       1|   32|        10|
|       1|   34|        10|
|       1|   35|        10|
+--------+-----+----------+



group by range

In [9]:
val result = unitsByRange
                .groupBy($"Category", $"rangeStart")
                .agg(sum("Value").alias("summing"))
                .orderBy("rangeStart")
                .drop("rangeStart")

result: org.apache.spark.sql.DataFrame = [Category: int, summing: bigint]


In [10]:
result.show(false)

+--------+-------+
|Category|summing|
+--------+-------+
|1       |40     |
|1       |103    |
|1       |101    |
+--------+-------+



# 2 - Group By (And sum) consecutive Time Intervals

In [11]:
import org.apache.spark.sql.functions
import org.apache.spark.sql.expressions.Window

import org.apache.spark.sql.functions
import org.apache.spark.sql.expressions.Window


In [12]:
val df = Seq((0, "2016-07-02 12:01:40"),
 (1, "2016-07-02 12:21:23"),
 (1, "2016-07-02 13:22:56"),
 (1, "2016-07-02 13:27:07"),
 (0, "2016-07-02 13:30:12"),
 (0, "2016-07-02 13:40:34"),
 (1, "2016-07-02 13:57:07"),
 (1, "2016-07-02 14:08:07")).
toDF("signal", "timestamp").
withColumn("timestamp", functions.to_timestamp(col("timestamp"), "yyyy-MM-dd HH:mm:ss"))
df.show()

+------+-------------------+
|signal|          timestamp|
+------+-------------------+
|     0|2016-07-02 12:01:40|
|     1|2016-07-02 12:21:23|
|     1|2016-07-02 13:22:56|
|     1|2016-07-02 13:27:07|
|     0|2016-07-02 13:30:12|
|     0|2016-07-02 13:40:34|
|     1|2016-07-02 13:57:07|
|     1|2016-07-02 14:08:07|
+------+-------------------+



df: org.apache.spark.sql.DataFrame = [signal: int, timestamp: timestamp]


In [13]:
val newSignalWindow = Window.orderBy("timestamp")

newSignalWindow: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@20513eb3


In [14]:
val dfWithNewSignal = df.withColumn("new_signal",
                                    (functions.lag(col("signal"),1,0).over(newSignalWindow) === 0 && 
                                     col("signal") === 1).cast("bigint"))

dfWithNewSignal: org.apache.spark.sql.DataFrame = [signal: int, timestamp: timestamp ... 1 more field]


In [15]:
dfWithNewSignal.show

2019-11-06 16:47:27 WARN  WindowExec:66 - No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
+------+-------------------+----------+
|signal|          timestamp|new_signal|
+------+-------------------+----------+
|     0|2016-07-02 12:01:40|         0|
|     1|2016-07-02 12:21:23|         1|
|     1|2016-07-02 13:22:56|         0|
|     1|2016-07-02 13:27:07|         0|
|     0|2016-07-02 13:30:12|         0|
|     0|2016-07-02 13:40:34|         0|
|     1|2016-07-02 13:57:07|         1|
|     1|2016-07-02 14:08:07|         0|
+------+-------------------+----------+



In [16]:
val dfWithIdSignal = dfWithNewSignal.
                            filter(col("signal") === 1).
                            withColumn("new_signal", 
                                       functions.
                                          sum("new_signal").
                                          over(newSignalWindow))

dfWithIdSignal: org.apache.spark.sql.DataFrame = [signal: int, timestamp: timestamp ... 1 more field]


In [17]:
dfWithIdSignal.show

2019-11-06 16:47:29 WARN  WindowExec:66 - No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2019-11-06 16:47:29 WARN  WindowExec:66 - No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
+------+-------------------+----------+
|signal|          timestamp|new_signal|
+------+-------------------+----------+
|     1|2016-07-02 12:21:23|         1|
|     1|2016-07-02 13:22:56|         1|
|     1|2016-07-02 13:27:07|         1|
|     1|2016-07-02 13:57:07|         2|
|     1|2016-07-02 14:08:07|         2|
+------+-------------------+----------+



In [18]:
val resultDF = dfWithIdSignal.
                  groupBy("new_signal").
                      agg(functions.min("timestamp").as("start_date"), functions.max("timestamp").as("end_date"),
                          functions.count("*").as("positive_count")).drop("new_signal")

resultDF: org.apache.spark.sql.DataFrame = [start_date: timestamp, end_date: timestamp ... 1 more field]


In [19]:
resultDF.show

2019-11-06 16:47:29 WARN  WindowExec:66 - No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2019-11-06 16:47:29 WARN  WindowExec:66 - No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
+-------------------+-------------------+--------------+
|         start_date|           end_date|positive_count|
+-------------------+-------------------+--------------+
|2016-07-02 12:21:23|2016-07-02 13:27:07|             3|
|2016-07-02 13:57:07|2016-07-02 14:08:07|             2|
+-------------------+-------------------+--------------+



# 3 - Count Occurence

In [20]:
val df = Seq(
  (1, "4/8/2019", "CLM***120379893***John***CLM***Smith***blablabla**so..on…"),
  (2, "4/8/2019", "CLM***120379093***John***CLM***Smith***CLM***blablabla**so..on…"),
  (3, "4/8/2019", "CLM***139979893***John***Smith***blablabla**so..on…")
).toDF("id", "date", "content")

df: org.apache.spark.sql.DataFrame = [id: int, date: string ... 1 more field]


In [21]:
def countAll(pattern: String) = udf((s: String) => pattern.r.findAllIn(s).size)

countAll: (pattern: String)org.apache.spark.sql.expressions.UserDefinedFunction


In [22]:
df.withColumn("wordcount", countAll("CLM")($"content")).show

+---+--------+--------------------+---------+
| id|    date|             content|wordcount|
+---+--------+--------------------+---------+
|  1|4/8/2019|CLM***120379893**...|        2|
|  2|4/8/2019|CLM***120379093**...|        3|
|  3|4/8/2019|CLM***139979893**...|        1|
+---+--------+--------------------+---------+



# 4 - Check if string exist in a column of another dataframe

In [23]:
  val df1 = Seq(
    (1, "google.com"),
    (2, "facebook.com"),
    (3, "github.com"),
    (4, "stackoverflow.com")).toDF("id", "url").as("first")
  df1.show

+---+-----------------+
| id|              url|
+---+-----------------+
|  1|       google.com|
|  2|     facebook.com|
|  3|       github.com|
|  4|stackoverflow.com|
+---+-----------------+



df1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: int, url: string]


In [24]:
  val df2 = Seq(
    (11, "google.com"),
    (12, "yahoo.com"),
    (13, "facebook.com"),
    (14, "twitter.com")).toDF("id", "url").as("second")
  df2.show

+---+------------+
| id|         url|
+---+------------+
| 11|  google.com|
| 12|   yahoo.com|
| 13|facebook.com|
| 14| twitter.com|
+---+------------+



df2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: int, url: string]


In [25]:
import spark.implicits._
import org.apache.spark.sql.functions.{col, _}

import spark.implicits._
import org.apache.spark.sql.functions.{col, _}


In [26]:
val df3 = df2.join(df1, expr("first.url like second.url"), "full_outer")
                        .select(col("first.url"),col("first.url")
                                .contains(col("second.url")).as("check"))
                                    .filter("url is not null")

df3: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [url: string, check: boolean]


In [27]:
df3.show()

+-----------------+-----+
|              url|check|
+-----------------+-----+
|       google.com| true|
|     facebook.com| true|
|       github.com| null|
|stackoverflow.com| null|
+-----------------+-----+



In [28]:
df3.na.fill(Map("check" -> false)).show()

+-----------------+-----+
|              url|check|
+-----------------+-----+
|       google.com| true|
|     facebook.com| true|
|       github.com|false|
|stackoverflow.com|false|
+-----------------+-----+



# 5 - Spark Higher Order to compute Top N

In [29]:
 import org.apache.spark.sql._

import org.apache.spark.sql._


In [30]:
  val df = Seq(
      ("1", "p1,p1,p1,p1,p1,p3,p3,p2,p2,p2,p2"),
      ("2", "p2,p2,p2,p2,p2,p4,p4,p4,p1,p3")
    ).toDF("id", "product_list")

df: org.apache.spark.sql.DataFrame = [id: string, product_list: string]


In [31]:
df.show()

+---+--------------------+
| id|        product_list|
+---+--------------------+
|  1|p1,p1,p1,p1,p1,p3...|
|  2|p2,p2,p2,p2,p2,p4...|
+---+--------------------+



In [32]:
def getMetrics(value: Row, n: Int): (String, String) = {
    
    val split = value.getAs[String]("product_list").split(",")
    
    val sortedRecords = split.groupBy(x => x).map(data => (data._1, data._2.size)).toList.sortWith(_._2 > _._2)
    (value.getAs[String]("id"), sortedRecords.take(n).map(_._1).mkString(","))
}

getMetrics: (value: org.apache.spark.sql.Row, n: Int)(String, String)


In [33]:
val final_df = df.map(value =>
                     getMetrics(value,2)
                      ).withColumnRenamed("_1", "id")
                       .withColumnRenamed("_2", "most_seen_products")

final_df: org.apache.spark.sql.DataFrame = [id: string, most_seen_products: string]


In [34]:
final_df.show

+---+------------------+
| id|most_seen_products|
+---+------------------+
|  1|             p1,p2|
|  2|             p2,p4|
+---+------------------+

